In [60]:
import tensorflow as tf
from tensorflow.contrib import slim
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import os
from PIL import Image
from collections import OrderedDict

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
z = np.ones((1, 100)) * 0.5#np.random.randn(1, 100)

In [4]:
raw_data = np.load("datasets/omniglot_data.npy")

In [5]:
inp = np.ones((1, 28, 28, 1))
# inp = np.array([raw_data[1200][0]], dtype="float64")

In [6]:
from dagan_architectures import UResNetGenerator, Discriminator

g = UResNetGenerator(batch_size=1, layer_sizes=[64, 64, 128, 128], layer_padding=None,
                        inner_layers=[3] * 4, name="generator")

In [7]:
z_inp = tf.placeholder(tf.float32, [1,100], 'z-inp')
cond_inp = tf.placeholder(tf.float32, [1,28,28,1], 'cond-inp')

In [8]:
result = g(z_inp, cond_inp, training=False)


Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.Dense instead.
Tensor("generator/vector_expansion/Reshape:0", shape=(1, 2, 2, 8), dtype=float32)
Tensor("generator/vector_expansion/Reshape_1:0", shape=(1, 4, 4, 4), dtype=float32)
Tensor("generator/vector_expansion/Reshape_2:0", shape=(1, 7, 7, 2), dtype=float32)
Tensor("generator/vector_expansion/Reshape_3:0", shape=(1, 14, 14, 1), dtype=float32)

Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.


generator_total_layers 50
generator_parameter_num 10546217


In [9]:
checkpoint = "checkpoints/checkpoint.ckpt"
fine_tune = slim.assign_from_checkpoint_fn(
    checkpoint,
    tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

sess = tf.Session()
fine_tune(sess)

INFO:tensorflow:Restoring parameters from checkpoints/checkpoint.ckpt


In [10]:
final_results = sess.run(
    result,
    feed_dict={z_inp: z, cond_inp: inp},
)
final_results[0][0, :, :, 0][5][10]

-0.4691119

In [53]:
def _conv2d(in_channels, out_channels, kernel_size, stride, activate=True, dropout=0.0):
    layers = OrderedDict()
    if stride == 2:
        padding = 0
        layers['pad'] = nn.ZeroPad2d((0, 1, 0, 1))
    else:
        padding = 1
    layers['conv'] = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
    if activate:
        layers['relu'] = nn.LeakyReLU(0.2)
        layers['batchnorm'] = nn.BatchNorm2d(out_channels, eps=1e-3, momentum=0.01)
    
    if dropout > 0.0:
        layers['dropout'] = nn.Dropout(dropout)
    return nn.Sequential(layers)


class _EncoderBlock(nn.Module):
    def __init__(
        self, pre_channels, in_channels, out_channels, num_layers, dropout_rate=0.0
    ):
        super().__init__()
        self.num_layers = num_layers
        self.pre_conv = _conv2d(
            in_channels=pre_channels, out_channels=pre_channels, kernel_size=3, stride=2, activate=False
        )

        self.conv0 = _conv2d(
            in_channels=in_channels + pre_channels, out_channels=out_channels, kernel_size=3, stride=1
        )
        total_channels = in_channels + out_channels
        for i in range(1, num_layers):
            self.add_module(
                'conv%d' % i,
                _conv2d(
                    in_channels=total_channels, out_channels=out_channels, kernel_size=3, stride=1
                )
            )
            total_channels += out_channels
        self.add_module(
            'conv%d' % num_layers,
            _conv2d(
                in_channels=total_channels, out_channels=out_channels, kernel_size=3, stride=2, dropout=dropout_rate
            )
        )

    def forward(self, inp):
        pre_input, x = inp
        if pre_input.shape[2] % 2 == 1:
            pre_input = F.pad(pre_input, (1, 0, 1, 0))
        pre_input = self.pre_conv(pre_input)
        out = self.conv0(torch.cat([x, pre_input], 1))
#         print(out[0][0])
        
        all_outputs = [x, out]
        for i in range(1, self.num_layers + 1):
            input_features = torch.cat([all_outputs[-1], all_outputs[-2]] + all_outputs[:-2], 1)
            module = self._modules['conv%d' % i]
            if input_features.shape[2] % 2 == 1 and module.conv.stride[0] == 2:
                input_features = F.pad(input_features, (1, 0, 1, 0))
            out = module(input_features)
            all_outputs.append(out)
#         print([o[0][0] for o in all_outputs])
        return all_outputs[-2], all_outputs[-1]


class TorchGAN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_sizes = [64, 64, 128, 128]
        self.num_inner_layers = 3
        self.conv0 = _conv2d(
            in_channels=1, out_channels=self.layer_sizes[0], kernel_size=3, stride=2
        )
        for i in range(1, len(self.layer_sizes)):
            self.add_module(
                "encode%d" % i,
                _EncoderBlock(
                    self.layer_sizes[i - 1], self.layer_sizes[i - 1], self.layer_sizes[i], self.num_inner_layers
                )
            )

    def forward(self, x):
        out = [x, self.conv0(x)]
        for i in range(1, len(self.layer_sizes)):
            out = self._modules["encode%d" % i](out)
            print(out[0].shape, out[1].shape)
#             if i == 1:
#                 print(out[1][0][0])
#         out = self.encode1(out)
#         out = self.act(out)
#         out = self.bn(out)
        return out[1]

In [54]:
torch_g = TorchGAN()
torch_g.eval()

TorchGAN(
  (conv0): Sequential(
    (pad): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    (conv): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2))
    (relu): LeakyReLU(negative_slope=0.2)
    (batchnorm): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  )
  (encode1): _EncoderBlock(
    (pre_conv): Sequential(
      (pad): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2))
    )
    (conv0): Sequential(
      (conv): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu): LeakyReLU(negative_slope=0.2)
      (batchnorm): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    )
    (conv1): Sequential(
      (conv): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu): LeakyReLU(negative_slope=0.2)
      (batchnorm): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    )
    (conv2): S

In [55]:
ckpt_reader = tf.train.load_checkpoint(checkpoint)
def read_tf(var):
    return torch.tensor(ckpt_reader.get_tensor(var))

def reshape_conv(tensor):
    return tensor.transpose(0, 3).transpose(1, 2).transpose(2, 3)

In [56]:
def convert_conv_layer(torch_layer, conv_path, bn_path):
    torch_layer.batchnorm.weight.data = read_tf(bn_path + "/gamma")
    torch_layer.batchnorm.bias.data = read_tf(bn_path + "/beta")
    torch_layer.batchnorm.running_mean = read_tf(bn_path + "/moving_mean")
    torch_layer.batchnorm.running_var = read_tf(bn_path + "/moving_variance")
    
    torch_layer.conv.weight.data = reshape_conv(read_tf(conv_path + "/kernel"))
    torch_layer.conv.bias.data = read_tf(conv_path + "/bias")

convert_conv_layer(
    torch_g.conv0, "generator/conv_layers/g_conv0/conv2d", "generator/conv_layers/g_conv0/BatchNorm")

for i in range(1, 4):
    module = torch_g._modules["encode%d" % i]
    module.pre_conv.conv.weight.data = reshape_conv(read_tf(f"generator/conv_layers/g_conv{i}/conv2d/kernel"))
    module.pre_conv.conv.bias.data = read_tf(f"generator/conv_layers/g_conv{i}/conv2d/bias")

    for j in range(4):
        bn_suffix = f"_{j}" if j > 0 else ""
        convert_conv_layer(
            module._modules[f"conv{j}"],
            f"generator/conv_layers/g_conv{i}/conv2d_{j + 1}",
            f"generator/conv_layers/g_conv{i}/BatchNorm{bn_suffix}",
        )

In [61]:
# sum(sum(torch_g.conv.weight[0][0]))
# print(torch_g(torch.ones(1, 1, 28, 28)).shape)
torch_g(torch.ones(1, 1, 28, 28))[0][0]

torch.Size([1, 64, 14, 14]) torch.Size([1, 64, 7, 7])
torch.Size([1, 128, 7, 7]) torch.Size([1, 128, 4, 4])
torch.Size([1, 128, 4, 4]) torch.Size([1, 128, 2, 2])


tensor([[-0.5380, -0.3327],
        [ 0.2860, -0.3094]], grad_fn=<SelectBackward>)

In [64]:
final_results[3][13][0, :, :, 0]

array([[-0.5379593 , -0.33272868],
       [ 0.28598887, -0.30941498]], dtype=float32)

In [ ]:
def render_image_arr(arr):
    arr = np.uint8(arr * 256)
    arr = arr.reshape(arr.shape[:-1])
    display(Image.fromarray(arr, mode='L').resize((224, 224)))

In [ ]:
vars_list = tf.train.list_variables(checkpoint)
vars_list

In [ ]:
torch_g.conv._modules['0']

In [ ]:
render_image_arr(raw_data[1200][0])
render_image_arr(final_results[0][0])

In [ ]:
tf.image.resize_nearest_neighbor(inp, (None, None))